# **03 - Feature Engineering**

In [1]:
import sys
sys.path.insert(0, "../src")

import pandas as pd
import numpy as np
from dataset import Dataset

pd.set_option("display.max_columns", None)

## **Normalization**

### **Verify normalization across all years**

Each year must produce the exact same column set regardless of its format group.

In [2]:
from dataset import (
    CANONICAL_CHARACTERISTICS,
    CANONICAL_LOCATIONS,
    CANONICAL_VEHICLES,
    CANONICAL_USERS,
)

for y in list(range(2010, 2023)) + ["test"]:
    ds = Dataset(y)
    assert ds.characteristics.columns.tolist() == CANONICAL_CHARACTERISTICS, f"{y}: characteristics columns mismatch"
    assert ds.locations.columns.tolist() == CANONICAL_LOCATIONS, f"{y}: locations columns mismatch"
    assert ds.vehicles.columns.tolist() == CANONICAL_VEHICLES, f"{y}: vehicles columns mismatch"
    assert ds.users.columns.tolist() == CANONICAL_USERS, f"{y}: users columns mismatch"
    n_acc = ds.characteristics["Num_Acc"].nunique()
    print(f"{str(y):5s} (group {ds.group}) : {n_acc:>6,} accidents  OK")

print("\nAll years normalized successfully.")

2010  (group A) : 69,379 accidents  OK
2011  (group A) : 66,974 accidents  OK
2012  (group B) : 56,025 accidents  OK
2013  (group B) : 52,558 accidents  OK
2014  (group B) : 53,869 accidents  OK
2015  (group B) : 52,789 accidents  OK
2016  (group B) : 53,489 accidents  OK
2017  (group B) : 54,631 accidents  OK
2018  (group B) : 52,005 accidents  OK
2019  (group C) : 52,956 accidents  OK
2020  (group C) : 42,970 accidents  OK
2021  (group C) : 50,867 accidents  OK
2022  (group C) : 49,772 accidents  OK
test  (group D) : 63,544 accidents  OK

All years normalized successfully.


### **Build full training set**

In [3]:
train = pd.concat(
    [Dataset(y).merged() for y in range(2010, 2023)],
    ignore_index=True,
)
print(f"Full training set: {train.shape[0]:,} rows x {train.shape[1]} cols")
print(f"Unique accidents:  {train['Num_Acc'].nunique():,}")
train.head()

Full training set: 1,583,848 rows x 46 cols
Unique accidents:  708,284


,Num_Acc,an,mois,jour,hrmn,lum,agg,int,atm,col,com,dep,catr,voie,v1,v2,circ,nbv,pr,pr1,vosp,prof,plan,lartpc,larrout,surf,infra,situ,num_veh,senc,catv,occutc,obs,obsm,choc,manv,place,catu,grav,sexe,an_nais,trajet,secu1,locp,actp,etatp
0,201000000001,2010,6,12,1930,1,2,1,1,6,52,590,3,39.0,NaN,NaN,2,2,0.0,0.0,0,1,1,0.0,60.0,1,0,1,A01,0,30,0,1,0,1,1,1,1,3,1,1976,5,2,<NA>,<NA>,<NA>
1,201000000002,2010,8,7,1000,1,2,1,1,6,477,590,3,39.0,NaN,NaN,2,0,0.0,0.0,0,1,1,0.0,0.0,1,0,1,A01,0,7,0,0,1,1,1,1,1,1,2,1983,5,1,<NA>,<NA>,<NA>
2,201000000002,2010,8,7,1000,1,2,1,1,6,477,590,3,39.0,NaN,NaN,2,0,0.0,0.0,0,1,1,0.0,0.0,1,0,1,A01,0,7,0,0,1,1,1,<NA>,3,3,1,1965,<NA>,<NA>,4,3,1
3,201000000003,2010,9,11,1600,1,1,1,1,5,11,590,3,41.0,2.0,NaN,2,2,0.0,0.0,0,1,1,0.0,61.0,1,0,1,A01,0,33,0,0,2,1,17,1,1,3,1,1979,5,<NA>,<NA>,<NA>,<NA>
4,201000000003,2010,9,11,1600,1,1,1,1,5,11,590,3,41.0,2.0,NaN,2,2,0.0,0.0,0,1,1,0.0,61.0,1,0,1,C03,0,7,0,0,9,3,23,1,1,1,2,1959,5,1,<NA>,<NA>,<NA>


## **Feature Engineering**

### **Target variable: GRAVE**

Create a binary target at the **accident level**: `GRAVE = 1` if at least one user was killed (`grav=2`) or hospitalized (`grav=3`), `0` otherwise. We use `groupby("Num_Acc").transform("max")` to propagate the label to every row of the same accident.

In [4]:
train['grav'].value_counts()

grav
1    653242
4    586816
3    301107
2     42683
Name: count, dtype: Int64

In [5]:
train["GRAVE"] = train["grav"].isin([2, 3]).astype(int)
train["GRAVE"] = train.groupby("Num_Acc")["GRAVE"].transform("max")
train.drop(columns=["grav"], inplace=True)
train["GRAVE"].value_counts()

GRAVE
0    930831
1    653017
Name: count, dtype: int64

### **Accident-level aggregation features**

Count features derived from `groupby("Num_Acc")`:
- `nb_vehicules` — number of distinct vehicles involved
- `nb_usagers` — total number of users (drivers, passengers, pedestrians)
- `nb_pietons` — number of pedestrians (`catu == 3`)
- `nb_occupants_vehicule` — number of non-pedestrian users per vehicle (`catu != 3`), to avoid counting pedestrians who are linked to a vehicle only because the accident involves them

In [6]:
# Per accident
train["nb_vehicules"] = train.groupby("Num_Acc")["num_veh"].transform("nunique")
train["nb_usagers"]   = train.groupby("Num_Acc")["catu"].transform("count")
train["nb_pietons"]   = train.groupby("Num_Acc")["catu"].transform(lambda x: (x == 3).sum())

# Per vehicle: count only non-pedestrians (catu != 3)
occupants = (
    train[train["catu"] != 3]
    .groupby(["Num_Acc", "num_veh"])
    .size()
    .rename("nb_occupants_vehicule")
)
train = train.merge(occupants, on=["Num_Acc", "num_veh"], how="left")
train["nb_occupants_vehicule"] = train["nb_occupants_vehicule"].fillna(0).astype(int)

train[["nb_vehicules", "nb_usagers", "nb_pietons", "nb_occupants_vehicule"]].describe()

,nb_vehicules,nb_usagers,nb_pietons,nb_occupants_vehicule
count,1.583848e+06,1583848.0,1.583848e+06,1.583848e+06
mean,1.881821e+00,2.85573,1.889809e-01,1.549265e+00
std,9.473748e-01,2.301026,4.896908e-01,1.747242e+00
min,1.000000e+00,1.0,0.000000e+00,0.000000e+00
25%,1.000000e+00,2.0,0.000000e+00,1.000000e+00
50%,2.000000e+00,2.0,0.000000e+00,1.000000e+00
75%,2.000000e+00,3.0,0.000000e+00,2.000000e+00
max,5.400000e+01,68.0,2.500000e+01,6.800000e+01


### **Individual-level features**

Derive per-user attributes from existing columns:
- `age` — age at the time of the accident (`an - an_nais`). Values outside [0, 120] are set to NaN as data entry errors.

In [7]:
train["age"] = train["an"] - train["an_nais"]
train.loc[(train["age"] < 0) | (train["age"] > 120), "age"] = np.nan

print(f"age NaN: {train['age'].isna().sum():,} ({train['age'].isna().mean():.2%})")
train["age"].describe()

age NaN: 5,378 (0.34%)


count    1578470.0
mean     38.232729
std       18.61174
min            0.0
25%           23.0
50%           35.0
75%           51.0
max          120.0
Name: age, dtype: Float64

### **Drop low-value columns**

Remove free-text, identifier, and high-cardinality string columns that carry no direct predictive signal for a tree-based model:

| Column | Reason |
|--------|--------|
| `voie` | Road name/number — free-text, very high cardinality |
| `v1`, `v2` | Alphanumeric road index points — free-text identifiers |
| `pr`, `pr1` | Milestone references (bornes kilométriques) — numeric road markers, not meaningful as features |
| `com` | Commune code — inconsistent format across groups (3-digit vs full INSEE), high cardinality (~36k unique). Geographic info already captured by `dep` |

Note: `adr`, `lat`, `long` were already dropped during normalization (free-text address and incompatible coordinate systems).

In [8]:
drop_cols = ["voie", "v1", "v2", "pr", "pr1", "com"]
train.drop(columns=drop_cols, inplace=True)

print(f"Dropped {len(drop_cols)} columns: {drop_cols}")
print(f"Remaining: {train.shape[1]} columns")
train.columns.tolist()

Dropped 6 columns: ['voie', 'v1', 'v2', 'pr', 'pr1', 'com']
Remaining: 45 columns


['Num_Acc',
 'an',
 'mois',
 'jour',
 'hrmn',
 'lum',
 'agg',
 'int',
 'atm',
 'col',
 'dep',
 'catr',
 'circ',
 'nbv',
 'vosp',
 'prof',
 'plan',
 'lartpc',
 'larrout',
 'surf',
 'infra',
 'situ',
 'num_veh',
 'senc',
 'catv',
 'occutc',
 'obs',
 'obsm',
 'choc',
 'manv',
 'place',
 'catu',
 'sexe',
 'an_nais',
 'trajet',
 'secu1',
 'locp',
 'actp',
 'etatp',
 'GRAVE',
 'nb_vehicules',
 'nb_usagers',
 'nb_pietons',
 'nb_occupants_vehicule',
 'age']

### **Variable transformations**

- `hrmn` (integer HHMM) → extract `hour` (0-23), then drop `hrmn`
- `lartpc`, `larrout` — road widths stored as strings → convert to numeric
- `an_nais` — redundant with `age` → drop

In [9]:
# Extract hour from HHMM
train["hour"] = train["hrmn"] // 100

# Convert road widths to numeric
for col in ("lartpc", "larrout"):
    train[col] = pd.to_numeric(train[col], errors="coerce")

# Drop redundant columns
train.drop(columns=["hrmn", "an_nais"], inplace=True)

print(f"hour range: [{train['hour'].min()}, {train['hour'].max()}]")
print(f"lartpc dtype: {train['lartpc'].dtype}, larrout dtype: {train['larrout'].dtype}")
print(f"Remaining: {train.shape[1]} columns")

hour range: [0, 23]
lartpc dtype: float64, larrout dtype: float64
Remaining: 44 columns


### **Drop identifiers**

`Num_Acc` and `num_veh` are row identifiers with no predictive value. Dropped after all groupby-based features have been computed.

In [10]:
train.drop(columns=["Num_Acc", "num_veh"], inplace=True)

print(f"Final shape: {train.shape[0]:,} rows x {train.shape[1]} columns")
train.dtypes

Final shape: 1,583,848 rows x 42 columns


an                         Int16
mois                        Int8
jour                        Int8
lum                         Int8
agg                         Int8
int                         Int8
atm                         Int8
col                         Int8
dep                          str
catr                        Int8
circ                        Int8
nbv                         Int8
vosp                        Int8
prof                        Int8
plan                        Int8
lartpc                   float64
larrout                  float64
surf                        Int8
infra                       Int8
situ                        Int8
senc                        Int8
catv                       Int16
occutc                     Int16
obs                         Int8
obsm                        Int8
choc                        Int8
manv                        Int8
place                       Int8
catu                        Int8
sexe                        Int8
trajet    

In [11]:
train.head()

,an,mois,jour,lum,agg,int,atm,col,dep,catr,circ,nbv,vosp,prof,plan,lartpc,larrout,surf,infra,situ,senc,catv,occutc,obs,obsm,choc,manv,place,catu,sexe,trajet,secu1,locp,actp,etatp,GRAVE,nb_vehicules,nb_usagers,nb_pietons,nb_occupants_vehicule,age,hour
0,2010,6,12,1,2,1,1,6,590,3,2,2,0,1,1,0.0,60.0,1,0,1,0,30,0,1,0,1,1,1,1,1,5,2,<NA>,<NA>,<NA>,1,1,1,0,1,34,19
1,2010,8,7,1,2,1,1,6,590,3,2,0,0,1,1,0.0,0.0,1,0,1,0,7,0,0,1,1,1,1,1,2,5,1,<NA>,<NA>,<NA>,1,1,2,1,1,27,10
2,2010,8,7,1,2,1,1,6,590,3,2,0,0,1,1,0.0,0.0,1,0,1,0,7,0,0,1,1,1,<NA>,3,1,<NA>,<NA>,4,3,1,1,1,2,1,1,45,10
3,2010,9,11,1,1,1,1,5,590,3,2,2,0,1,1,0.0,61.0,1,0,1,0,33,0,0,2,1,17,1,1,1,5,<NA>,<NA>,<NA>,<NA>,1,3,3,0,1,31,16
4,2010,9,11,1,1,1,1,5,590,3,2,2,0,1,1,0.0,61.0,1,0,1,0,7,0,0,9,3,23,1,1,2,5,1,<NA>,<NA>,<NA>,1,3,3,0,1,51,16
